In [ ]:
import pandas as pd
import numpy as np
import quandl as qd
import matplotlib.pyplot as plt

In [ ]:
qd.ApiConfig.api_key = 'xrJsb6qPU-whbsegBjyE'

In [ ]:
#define industry and stocks
airlines = ['DAL','AAL','JBLU','LUV','SAVE','ALK','UAL','AER','AL']
banks = ['C','BAC','MS','JPM','GS','WFC','RY','TD','BNC']
casinos = ['MGM','WYNN','MLCO','BYD','LVS','CZR','ERI','PENN']
hotels = ['MAR','HLT','EXPE','TCOM','H','IHG','CHH','WH','WYND','PK']

In [ ]:
def single_stock(stock, choice):
    df = qd.get_table('SHARADAR/SF1',dimension='MRY',ticker=stock)
    if choice == 'income':
        return analyze_income(df)
    if choice == 'debt':
        return analyze_debt(df)
    
def analyze_debt(df):
    debt_cols = ['netinc','intexp','cashneq','assetsc','liabilitiesc','investmentsc','ncfdebt','reportperiod','lastupdated','opex','ticker']
    debt = (df.T.loc[debt_cols]).T
    debt['cash_ratio'] = debt['cashneq']/debt['liabilitiesc']
    debt['quick_ratio'] = (debt['cashneq'] + debt['investmentsc'])/debt['liabilitiesc']
    debt['current_ratio'] = debt['assetsc'] / debt['liabilitiesc']
    debt['cash_intexp'] = debt['cashneq'] / (debt['intexp']+1)
    debt['cash_allexp'] = debt['cashneq'] / (debt['intexp'] + debt['opex'])
    debt['cash_ncfdebt'] = debt['cashneq'] / (debt['ncfdebt'] + 1)
    debt['inc_intexp'] = debt['netinc']/(debt['intexp']+1)
    ret_cols = ['cash_ratio','quick_ratio','current_ratio','inc_intexp','cash_intexp','cash_allexp','cash_ncfdebt','reportperiod','ticker']
    return debt[ret_cols]

def analyze_income(df):
    inc_cols = ['revenue','netinc','sgna','rnd','opex','grossmargin','netmargin','ebitdamargin','roe','roa','reportperiod','ticker']
    inc = (df.T.loc[inc_cols]).T
    print(inc)
    inc['revenue_growth'] = (inc['revenue']-inc['revenue'].shift(-1))/inc['revenue'].shift(-1) * 100
    inc['income_growth'] = (inc['netinc']-inc['netinc'].shift(-1))/inc['netinc'].shift(-1) * 100
    ret_cols = inc_cols + ['revenue_growth','income_growth']
    return inc[ret_cols]

def get_data(industry):
    dfs = {'income':{}, 'debt': {}}
    for stock in industry:
        df = qd.get_table('SHARADAR/SF1',dimension='MRY',ticker=stock)
        tmp = analyze_debt(df)
        tmp['reportperiod'] = pd.to_datetime(tmp['reportperiod'])
        tmp = tmp.sort_values('reportperiod')
        tmp.index = tmp['reportperiod']
        dfs['debt'][stock] = tmp
        del tmp['reportperiod']
        
        tmp = analyze_income(df)
        tmp['reportperiod'] = pd.to_datetime(tmp['reportperiod'])
        tmp = tmp.sort_values('reportperiod')
        tmp.index = tmp['reportperiod']
        dfs['income'][stock] = tmp
        del tmp['reportperiod']
    return dfs


def analyze_industry_cross(dfs,choice,metric):
    tmp = pd.DataFrame()
    df = dfs[choice]
    for k in df.keys():
        tmp[k] = df[k][metric]
    tmp.plot(figsize=(20,10))
    plt.show()
    
def analyze_industry_current(dfs, choice):
#     tmp = {}
#     for k in dfs.keys():
#         print(dfs[k].iloc[0])
    df = dfs[choice]
    lst = [df[k] for k in df.keys()]
    tmp = pd.concat(lst)
    dates = tmp.index.unique()
    tmp = tmp[tmp.index==dates[-1]]
    tmp.index = tmp.ticker
    del tmp['ticker']
    print(tmp)

In [ ]:
dfs = get_data(hotels)

In [ ]:
analyze_industry_cross(dfs,'income','roe')

In [ ]:
analyze_industry_current(dfs,'debt')

In [ ]:
analyze_industry_current(dfs,'income')

In [ ]:
single_stock('PK','income')